# Brazilian Partition

TO DIVIDE BRAZILIAN MUNICIPALITIES BY POPULATION

In [ ]:
from mip import *

import re
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import itertools

### DATA

Map data

In [ ]:
gdf = gpd.read_file("Data/br_municipios_2018/BRMUE250GC_SIR.shp")
gdf = gdf.drop(gdf[(gdf["CD_GEOCMU"] == "4300001") | (gdf["CD_GEOCMU"] == "4300002")].index)
gdf = gdf.drop(gdf[(gdf["CD_GEOCMU"] == "2605459") | (gdf["CD_GEOCMU"] == "3520400")].index)
gdf = gdf.set_index("CD_GEOCMU")
gdf.head()

Population data

In [ ]:
xls = pd.ExcelFile('Data/estimativa_dou_2019.ods', engine="odf")

In [ ]:
pop = pd.read_excel(xls, 1, header=1).loc[:5569]
pop["CD_GEOCMU"] = pop[['COD. UF', 'COD. MUNIC']].apply(lambda x: str(int(x[0])) + str(int(x[1])).zfill(5), axis=1)

pop = pop[["CD_GEOCMU", "POPULAÇÃO ESTIMADA"]]
pop.columns = ["CD_GEOCMU", "POP"]

pop["POP"] = pop["POP"].apply(lambda x: int(re.sub(r'\([^)]*\)', '', str(x)).replace(".","")))
pop = pop.drop(pop[(pop["CD_GEOCMU"] == "2605459") | (pop["CD_GEOCMU"] == "3520400")].index)

pop = pop.set_index("CD_GEOCMU")
pop = pop.reindex(gdf.index)

pop.head()

Graph data

In [ ]:
g = nx.read_gml("Data/Brazil.gml")
print(nx.info(g))

Constants

In [ ]:
vals = pop["POP"].map(int).values.tolist()
pop_total = sum(vals)
cities = len(vals)
idx = gdf.index

Fake Constants

In [ ]:
cities = 25

codes = dict()
for i in range(cities):
    codes[i] = str(i)+"!"
idx = pd.Index([codes[i] for i in range(cities)])

g = nx.gnp_random_graph(cities, 5.7651/(cities-1))
g = nx.barbell_graph(int(.4*cities), cities-2*int(.4*cities))
g = nx.relabel_nodes(g, codes)

vals = np.round(np.random.normal(1000, 40, cities),0).tolist()
pop_total = sum(vals)

### A SINGLE CONNECTED REGION

[Connectivity](https://www.cc.gatech.edu/~bdilkina/papers/CPAIOR2010_dilkina.pdf)

In [ ]:
def set_connected(seed=None):
    edges = list(g.edges)
    A = edges + [e[::-1] for e in edges]
    N = g.number_of_nodes()
    
    if seed == None:
        seed = idx.get_loc("3550308")
    
    m = Model()
    
    x = [m.add_var(var_type=BINARY) for j in range(cities)]
    x0 = m.add_var(ub=N) 
    y = [m.add_var() for e in A]
    y0 = m.add_var()
    abs_diff = m.add_var()

    
    m += x0 + y0 == N

    
    for e in range(len(A)):
        m += y[e] <= N*x[idx.get_loc(A[e][1])]
    y0 <= N*x[seed]

    
    for j in range(1, cities):
        m += xsum(y[e] for e in range(len(A)) if idx.get_loc(A[e][1])==j)\
            == x[j] + xsum(y[e] for e in range(len(A)) if idx.get_loc(A[e][0])==j) 
    m += xsum(y[e] for e in range(len(A))if idx.get_loc(A[e][1])==seed) + y0\
        == x[seed] + xsum(y[e] for e in range(len(A)) if idx.get_loc(A[e][0])==seed)

    
    m += xsum(x[j] for j in range(cities)) == y0

    
    expected = int(round(pop_total / 2,0))
    m += xsum(x[j]*vals[j] for j in range(cities)) - expected <= abs_diff
    m += expected - xsum(x[j]*vals[j] for j in range(cities)) <= abs_diff

    m.objective = abs_diff
    
    print("Model set.")
    return m

### CONNECTED REGIONS

In [ ]:
def set_regions(n=2, seed=None):
    edges = list(g.edges)
    A = edges + [e[::-1] for e in edges]
    N = g.number_of_nodes()
    if seed == None and n < 4:
        seed = [idx.get_loc(x) for x in ["4305439", "1400704", "2408953", "3106200"]][:n]
    
    m = Model()
    
    x = [[m.add_var(var_type=BINARY) for j in range(cities)] for i in range(n)]
    x0 = [m.add_var(ub=N) for i in range(n)]
    y = [[m.add_var() for e in A] for i in range(n)]
    y0 = [m.add_var() for i in range(n)]
    abs_diff = [m.add_var() for i in range(n)]

    # Cada cidade tem uma, e somente uma, cor
    for j in range(cities):
        m += xsum(x[i][j] for i in range(n)) == 1
    
    for i in range(n):
        # Residual mais injetado é N
        m += x0[i] + y0[i] == N
        
        # Se tem fluxo nesta aresta, então o vértices está presente
        for e in range(len(A)):
            m += y[i][e] <= N*x[i][idx.get_loc(A[e][1])]
        y0[i] <= N*x[i][seed[i]]    
            
        # Fluxo que entra é o fluxo que sai mais o gasto pelo vértice
        for j in [k for k in range(cities) if k != seed[i]]:
            m += xsum(y[i][e] for e in range(len(A)) if idx.get_loc(A[e][1])==j)\
                == x[i][j] + xsum(y[i][e] for e in range(len(A)) if idx.get_loc(A[e][0])==j) 
        m += xsum(y[i][e] for e in range(len(A)) if idx.get_loc(A[e][1])==seed[i]) + y0[i]\
                == x[i][seed[i]] + xsum(y[i][e] for e in range(len(A)) if idx.get_loc(A[e][0])==seed[i]) 
        
        # Fluxo absorvido é o injetado
        m += xsum(x[i][j] for j in range(cities)) == y0[i]
    
    
    expected = int(round(pop_total / n,0))
    for i in range(n):
        m += xsum(x[i][j]*vals[j] for j in range(cities)) - expected <= abs_diff[i]
        m += expected - xsum(x[i][j]*vals[j] for j in range(cities)) <= abs_diff[i]

    m.objective = xsum(abs_diff)
    
    print("Model set.")
    return m

### MINIMIZES EDGES BETWEEN REGIONS

In [ ]:
def set_regions(n=2, coef=0.01, seed=None):
    edges = list(g.edges)
    A = edges + [e[::-1] for e in edges]
    N = g.number_of_nodes()
    C = list(itertools.combinations(range(n), 2))
    if seed == None and n < 4:
        seed = [idx.get_loc(x) for x in ["4305439", "1400704", "2408953", "3106200"]][:n]
    
    m = Model()
    
    x = [[m.add_var(var_type=BINARY) for j in range(cities)] for i in range(n)]
    x0 = [m.add_var(ub=N) for i in range(n)]
    y = [[m.add_var() for e in A] for i in range(n)]
    y0 = [m.add_var() for i in range(n)]
    c = [m.add_var() for ii in C]

    # Cada cidade tem uma, e somente uma, cor
    for j in range(cities):
        m += xsum(x[i][j] for i in range(n)) == 1
    
    for i in range(n):
        # Residual mais injetado é N
        m += x0[i] + y0[i] == N
        
        # Se tem fluxo nesta aresta, então o vértices está presente
        for e in range(len(A)):
            m += y[i][e] <= N*x[i][idx.get_loc(A[e][1])]
        y0[i] <= N*x[i][seed[i]]    
            
        # Fluxo que entra é o fluxo que sai mais o gasto pelo vértice
        for j in [k for k in range(cities) if k != seed[i]]:
            m += xsum(y[i][e] for e in range(len(A)) if idx.get_loc(A[e][1])==j)\
                == x[i][j] + xsum(y[i][e] for e in range(len(A)) if idx.get_loc(A[e][0])==j) 
        m += xsum(y[i][e] for e in range(len(A)) if idx.get_loc(A[e][1])==seed[i]) + y0[i]\
                == x[i][seed[i]] + xsum(y[i][e] for e in range(len(A)) if idx.get_loc(A[e][0])==seed[i]) 
        
        # Fluxo absorvido é o injetado
        m += xsum(x[i][j] for j in range(cities)) == y0[i]
    
    # População dentro da margem
    expected = pop_total / n
    for i in range(n):
        m += xsum(x[i][j]*vals[j] for j in range(cities)) <= expected * (1+coef)
        m += xsum(x[i][j]*vals[j] for j in range(cities)) >= expected * (1-coef)
        
    # ARESTAS ENTRE GRUPOS ??
    for ii in range(len(C)):
        i1 = c[ii][0]
        i2 = c[ii][1]
        m += xsum(x[i1][] + x[i2][] for e in edges) - len(edges) == c[ii]
            
    m.objective = xsum(c)
    
    print("Model set.")
    return m

### AUXILIARY FUNCTIONS

Solve function solves

In [ ]:
def solve(m, verbose=True):
    m.max_gap = 0.01
    m.emphasis = 1
    status = m.optimize()
    if verbose:
        if status == OptimizationStatus.OPTIMAL:
            print('optimal solution cost {} found'.format(m.objective_value))
        elif status == OptimizationStatus.FEASIBLE:
            print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
        elif status == OptimizationStatus.NO_SOLUTION_FOUND:
            print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))
        elif status == OptimizationStatus.INFEASIBLE:
            print('INFEASIBLE'.format(m.objective_bound))
    return m

Plot function

In [ ]:
def plot_map(colors):
    gdf["colors"] = colors
    
    fig, ax = plt.subplots(1, 1)
    ax.axis('off')
    fig.set_size_inches(20, 20)
    gdf.plot(column="colors", ax=ax, cmap='Accent')
    plt.show()

Get the color numbers from the MIP

In [ ]:
def get_colors_connected(m):
    dum = list()
    lw = 0
    up = cities

    return [j.x for j in m.vars[lw:up]]

Get the color numbers from the MIP

In [ ]:
def get_colors_regions(m, n=2):
    dum = list()
    lw = 0
    up = cities
    
    for i in range(n):
        dum.append([j.x for j in m.vars[lw:up]])
        lw += cities
        up += cities
    
    a = np.transpose(np.array(dum))
    return [np.where(r==1)[0][0] for r in a]

### RESULTS

Set seeds

In [ ]:
seed = [0, 100, 240]

Run the model

In [ ]:
m1 = set_regions()
m1 = solve(m1)

Plot the map

In [ ]:
plot_map(get_colors_connected(m1))

Plot fake data

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.axis('off')
fig.set_size_inches(10, 10)
nx.draw_networkx(g, node_color=get_colors_regions(m1), node_size=50, font_size=0, ax=ax)
plt.show()